In [ ]:
import sounddevice as sd
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

samplerate = 16000
channels = 1
buffer_size = 4096
bufferarray = np.array([])
current_time = time.time()


# ! To work, you must put your exact path to the downloaded model !
interpreter = tf.lite.Interpreter(model_path="model.tflite") # Loads the model for use
interpreter.allocate_tensors() # Gives the input and output tensors the resources needed to work

# Get input and output tensor details
input_details = interpreter.get_input_details() # Just gives you more information on the input data
output_details = interpreter.get_output_details() # Just gives you more information on the output data

def update_data_shape(data):
    data = data.squeeze()
    updated_data = tf.convert_to_tensor(data, dtype=tf.float32)
    data = updated_data[:31950]
    zero_padding = tf.zeros([31950] - tf.shape(data)[0], dtype=tf.float32)
    data = tf.concat([zero_padding, data],0)
    return data

def spectrogram_shape(data):
    spectrogram = tf.signal.stft(data, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    spectrogram = tf.expand_dims(spectrogram, axis=0)
    return spectrogram

def model_prediction(spectrogram):
    global current_time
    interpreter.set_tensor(input_details[0]['index'], spectrogram.numpy().astype(np.float32)) # Changes the input 
# data to be the spectrogram
    interpreter.invoke() # Predicts in the output space based on the updated input

    prediction = interpreter.get_tensor(output_details[0]['index']) # Getting that output from the model

    if prediction[0][0] > 0.8 and time.time() - current_time > 2:
        print("Clap!")
        current_time = time.time()

def callback(indata, frames, unknown, status):
    global bufferarray
    if status: print(status)
    bufferarray = np.append(bufferarray, indata.squeeze())
    if bufferarray.shape[0] + buffer_size >= 31950:
        model_prediction(spectrogram_shape(update_data_shape(bufferarray)))
        bufferarray = bufferarray[buffer_size:]

with sd.InputStream(channels=channels, callback=callback,
                         samplerate=samplerate, blocksize=buffer_size):
    try:
        while True:
            time.sleep(0.01)
    except KeyboardInterrupt:
        print("Stopping Recording")